# Natural Language Query Interface (NLQI)


1) Setup




In [ ]:
!pip -q install pandas duckdb sqlglot sentence-transformers faiss-cpu pydantic python-dateutil

2) Load sample data (replace with Unifier pulls)


In [ ]:
import pandas as pd, duckdb, json, datetime as dt
invoices = pd.DataFrame([
    {"invoice_id":1,"record_number":"INV-001","vendor_id":10,"contract_id":100,"invoice_total":150000,"tax_amount":22500,"currency":"SAR","status":"Pending","days_pending":14,"created_at":"2025-08-03"},
    {"invoice_id":2,"record_number":"INV-002","vendor_id":11,"contract_id":101,"invoice_total":45000,"tax_amount":6750,"currency":"SAR","status":"Approved","days_pending":2,"created_at":"2025-08-05"},
    {"invoice_id":3,"record_number":"INV-003","vendor_id":10,"contract_id":100,"invoice_total":220000,"tax_amount":33000,"currency":"SAR","status":"Pending","days_pending":21,"created_at":"2025-08-02"},
])
vendors = pd.DataFrame([
    {"vendor_id":10,"vendor_name":"ACME Construction"},
    {"vendor_id":11,"vendor_name":"Beta Supplies"}
])

con = duckdb.connect()
con.register("invoices", invoices)
con.register("vendors", vendors)
PROJECTS_VISIBLE_TO_USER = "TRUE"  # replace with project filter for RLS


3) Lightweight semantic mapper (embeddings → column picks)


In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer("all-MiniLM-L6-v2")
column_catalog = [
    ("invoices","invoice_total","amount total"),
    ("invoices","status","approval status"),
    ("invoices","days_pending","overdue pending age"),
    ("invoices","created_at","date created"),
    ("vendors","vendor_name","supplier name company"),
]
catalog_text = [f"{t}.{c}: {desc}" for t,c,desc in column_catalog]
catalog_emb = model.encode(catalog_text, normalize_embeddings=True)

def lookup(term, k=1):
    q = model.encode([term], normalize_embeddings=True)
    scores = util.cos_sim(q, catalog_emb).cpu().numpy()[0]
    idx = scores.argsort()[::-1][:k]
    return [column_catalog[i] for i in idx]


4) Simple intent parser (rule+heuristic for demo)


In [ ]:
import re
from dateutil.relativedelta import relativedelta

def parse_query(q):
    ql = q.lower()
    vendor = re.findall(r'vendor\s+([a-z0-9 ]+)', ql) or re.findall(r'\b(acme|beta)\b', ql)
    vendor_like = f"%{(vendor[0] if vendor else '').strip()}%"
    amt = re.findall(r'>(\s*\d+[kKmM]?)', ql)
    def to_number(x):
        x=x.strip().lower()
        return float(x[:-1])*1000 if x.endswith('k') else float(x[:-1])*1_000_000 if x.endswith('m') else float(x)
    min_amount = to_number(amt[0]) if amt else 0

    today = dt.date(2025,8,11)
    start = today.replace(day=1) if "this month" in ql else today - relativedelta(days=30)
    return {
        "vendor_like": vendor_like,
        "min_amount": min_amount,
        "from": str(start),
        "to": str(today),
        "limit": 100
    }

def build_sql(params):
    rls = PROJECTS_VISIBLE_TO_USER
    sql = f"""
    SELECT i.record_number, v.vendor_name, i.invoice_total, i.status, i.days_pending
    FROM invoices i
    JOIN vendors v ON i.vendor_id = v.vendor_id
    WHERE {rls}
      AND v.vendor_name ILIKE '{params["vendor_like"]}'
      AND i.invoice_total > {params["min_amount"]}
      AND i.status = 'Pending'
      AND i.created_at BETWEEN '{params["from"]}' AND '{params["to"]}'
    ORDER BY i.days_pending DESC
    LIMIT {params["limit"]}
    """
    return sql

q = "overdue invoices > 100k for vendor acme this month"
params = parse_query(q)
sql = build_sql(params)
res = con.execute(sql).df()
sql, res


5) Output contract
Return to the user:

The answer table

The generated SQL

Assumptions & RLS applied

A “Run in Unifier Report” hint with mapped fields